In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


# pd.options.display.max_rows
# pd.set_option('display.max_rows', None)


In [2]:
year = []
model = []
mileage = []
location = []
price = []
price_type = []
color_scheme = []
hist_cond = []

for i in range (1,31):
    
        # Website variable
        website = 'https://www.truecar.com/used-cars-for-sale/listings/bmw/3-series/year-2006-2011/location-effort-pa/?page=' + str(i) + '&searchRadius=5000&trimSlug[]=335i&trimSlug[]=335is&trimSlug[]=335xi'

        # Response to website
        response = requests.get(website)
        
        # Soup object
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Results
        results = soup.find_all('div', {'class':'linkable card card-shadow vehicle-card _1qd1muk'})
        
        for result in results:
            
            # year 
            try:
                year.append(result.find('span', {'class': 'vehicle-card-year font-size-1'}).get_text())
            except:
                year.append('n/a')
                
            # model
            try:
                model.append(result.find('div', {'class': 'font-size-1 text-truncate'}).get_text())
            except:
                model.append('n/a')
            
            # mileage
            try:
                mileage.append(result.find('div', {'data-test':'vehicleMileage'}).get_text())
            except:
                mileage.append('n/a')
            
            # location
            try:
                location.append(result.find('div', {'data-test':'vehicleCardLocation'}).get_text())
            except:
                location.append('n/a')
            
            # price
            try:
                price.append(result.find('div', {'data-test':'vehicleCardPricingBlockPrice'}).get_text())
            except:
                price.append('n/a')
            
            # price_type
            try:
                price_type.append(result.find('span', {'data-test':'graphIconLabel'}).get_text())
            except:
                price_type.append('n/a')
            
            # color_scheme
            try:
                color_scheme.append(result.find('div', {'data-test': 'vehicleCardColors'}).get_text())
            except:
                color_scheme.append('n/a')
            
            # hist_cond
            try:
                hist_cond.append(result.find('div', {'data-test':'vehicleCardCondition'}).get_text())
            except:
                hist_cond.append('n/a')

In [3]:
truecar_df = pd.DataFrame ({'Year':year, 'Model':model, 'Mileage':mileage,
                           'Location':location, 'Price':price, 'Site Price Type':price_type,
                           'Color Scheme':color_scheme, 'History':hist_cond})

In [4]:
truecar_df

,Year,Model,Mileage,Location,Price,Site Price Type,Color Scheme,History
0,2011,335is Convertible,"62,515 miles","58 mi - Somerset, NJ","$25,999",High Price,"Silver exterior, Black interior","No accidents, 3 Owners, Personal use"
1,2011,335i xDrive Coupe AWD,"37,739 miles","21 mi - Easton, PA","$25,998",Excellent Price,"White exterior, Brown interior","No accidents, 2 Owners, Personal use"
2,2011,328i xDrive Sedan AWD SULEV,"80,434 miles","74 mi - South Hackensack, NJ","$9,795",Excellent Price,"White exterior, Brown interior","No accidents, 2 Owners, Personal use"
3,2011,328i xDrive Sedan AWD,"92,100 miles","67 mi - Linden, NJ","$8,900",Excellent Price,"Gray exterior, Gray interior","No accidents, 4 Owners, Personal use"
4,2011,335i Sedan,"67,840 miles","8.7 mi - Stroudsburg, PA","$18,990",Great Price,"Blue exterior, Brown interior","No accidents, 5 Owners, Fleet use"
...,...,...,...,...,...,...,...,...
925,2011,328i xDrive Sedan AWD,"56,017 miles","73 mi - Hasbrouck Heights, NJ","$15,885",High Price,"Blue exterior, Brown interior","No accidents, 2 Owners, Personal use"
926,2011,328i xDrive Sedan AWD SULEV,"144,201 miles","52 mi - Somerville, NJ","$7,995",Excellent Price,"Unknown exterior, Unknown interior","1 accident, 4 Owners, Personal use"
927,2011,328i xDrive Sedan AWD SULEV,"119,730 miles","68 mi - Paterson, NJ","$11,486",High Price,"Black exterior, Black interior","No accidents, 1 Owner, Personal use"
928,2011,328i xDrive Sedan AWD,"129,862 miles","66 mi - Rahway, NJ","$9,900",High Price,"Unknown exterior, Unknown interior","1 accident, 4 Owners, Fleet use"


# Data Cleaning

In [5]:
truecar_df.columns

Index(['Year', 'Model', 'Mileage', 'Location', 'Price', 'Site Price Type',
       'Color Scheme', 'History'],
      dtype='object')

In [6]:
truecar_df.Model.value_counts()

328i xDrive Sedan AWD SULEV    210
335i Convertible               180
328i xDrive Sedan AWD          150
335i xDrive Coupe AWD           90
335is Convertible               60
335i Sedan                      60
328i Sedan                      60
328xi Sedan AWD SULEV           30
328xi Coupe AWD SULEV           30
325Ci Convertible               30
328i xDrive Coupe AWD           30
Name: Model, dtype: int64

### N54 Car Filter

In [7]:
truecar_df['335i_yn'] = truecar_df['Model'].apply(lambda x: 'yes' if '335i' in x.lower() else 'no')
truecar_df['335i_yn'].value_counts()

no     540
yes    390
Name: 335i_yn, dtype: int64

In [8]:
truecar_df.drop(truecar_df[truecar_df['335i_yn'] == 'no'].index, inplace = True)

In [9]:
truecar_df['Model'].value_counts()

335i Convertible         180
335i xDrive Coupe AWD     90
335is Convertible         60
335i Sedan                60
Name: Model, dtype: int64

### xDrive (All wheel drive)

In [10]:
truecar_df['xdrive_yn'] = truecar_df['Model'].apply(lambda x: 'yes' if 'awd' in x.lower() else 'no')

In [11]:
truecar_df['xdrive_yn'].value_counts()

no     300
yes     90
Name: xdrive_yn, dtype: int64

### 'is' model

In [12]:
truecar_df['is_model_yn'] = truecar_df['Model'].apply(lambda x: 'yes' if 'is' in x.lower() else 'no')

In [13]:
truecar_df['is_model_yn'].value_counts()

no     330
yes     60
Name: is_model_yn, dtype: int64

### E90 or E92

In [14]:
truecar_df['body_style'] = truecar_df['Model'].apply(lambda x: 'E90' if 'sedan' in x.lower() else 'E92')

In [15]:
truecar_df['body_style'].value_counts()

E92    330
E90     60
Name: body_style, dtype: int64

### Vehicle Color

In [16]:
truecar_df['car_color'] = truecar_df['Color Scheme'].apply(lambda x: x.split(' ')[0])

In [17]:
truecar_df['car_color'].value_counts()

Black      150
Blue       120
White       60
Silver      30
Unknown     30
Name: car_color, dtype: int64

In [18]:
truecar_df.head(25)

,Year,Model,Mileage,Location,Price,Site Price Type,Color Scheme,History,335i_yn,xdrive_yn,is_model_yn,body_style,car_color
0,2011,335is Convertible,"62,515 miles","58 mi - Somerset, NJ","$25,999",High Price,"Silver exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,yes,E92,Silver
1,2011,335i xDrive Coupe AWD,"37,739 miles","21 mi - Easton, PA","$25,998",Excellent Price,"White exterior, Brown interior","No accidents, 2 Owners, Personal use",yes,yes,no,E92,White
4,2011,335i Sedan,"67,840 miles","8.7 mi - Stroudsburg, PA","$18,990",Great Price,"Blue exterior, Brown interior","No accidents, 5 Owners, Fleet use",yes,no,no,E90,Blue
8,2011,335i Convertible,"55,578 miles","21 mi - Easton, PA","$23,998",High Price,"Black exterior, Unknown interior","No accidents, 2 Owners, Personal use",yes,no,no,E92,Black
11,2011,335i Convertible,"71,447 miles","8.7 mi - Stroudsburg, PA","$21,990",High Price,"Black exterior, Brown interior","No accidents, 6 Owners, Personal use",yes,no,no,E92,Black
15,2010,335i Convertible,"74,799 miles","8.7 mi - Stroudsburg, PA","$18,990",High Price,"Black exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,no,E92,Black
17,2011,335is Convertible,"40,262 miles","8.7 mi - Stroudsburg, PA","$28,990",High Price,"Blue exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,yes,E92,Blue
18,2011,335i xDrive Coupe AWD,"86,645 miles","67 mi - Elizabeth, NJ","$19,895",Fair Price,"White exterior, Red interior","No accidents, 1 Owner, Personal use",yes,yes,no,E92,White
20,2011,335i xDrive Coupe AWD,"34,908 miles","74 mi - Jersey City, NJ","$19,998",Excellent Price,"Blue exterior, Unknown interior","No accidents, 2 Owners, Personal use",yes,yes,no,E92,Blue
21,2011,335i Convertible,"89,479 miles","67 mi - Elizabeth, NJ","$17,495",Fair Price,"Blue exterior, Beige interior","1 accident, 2 Owners, Personal use",yes,no,no,E92,Blue


### Interior Color

In [19]:
truecar_df['interior_color'] = truecar_df['Color Scheme'].apply(lambda x: x.split(',')[1])

In [20]:
truecar_df['interior_color'].value_counts()

 Brown interior      120
 Black interior       90
 Unknown interior     90
 Beige interior       60
 Red interior         30
Name: interior_color, dtype: int64

In [21]:
truecar_df['interior_color'] = truecar_df['interior_color'].apply(lambda x: x.replace('interior', ''))

In [22]:
truecar_df['interior_color'].value_counts()

 Brown       120
 Black        90
 Unknown      90
 Beige        60
 Red          30
Name: interior_color, dtype: int64

In [23]:
truecar_df.head(10)

,Year,Model,Mileage,Location,Price,Site Price Type,Color Scheme,History,335i_yn,xdrive_yn,is_model_yn,body_style,car_color,interior_color
0,2011,335is Convertible,"62,515 miles","58 mi - Somerset, NJ","$25,999",High Price,"Silver exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,yes,E92,Silver,Black
1,2011,335i xDrive Coupe AWD,"37,739 miles","21 mi - Easton, PA","$25,998",Excellent Price,"White exterior, Brown interior","No accidents, 2 Owners, Personal use",yes,yes,no,E92,White,Brown
4,2011,335i Sedan,"67,840 miles","8.7 mi - Stroudsburg, PA","$18,990",Great Price,"Blue exterior, Brown interior","No accidents, 5 Owners, Fleet use",yes,no,no,E90,Blue,Brown
8,2011,335i Convertible,"55,578 miles","21 mi - Easton, PA","$23,998",High Price,"Black exterior, Unknown interior","No accidents, 2 Owners, Personal use",yes,no,no,E92,Black,Unknown
11,2011,335i Convertible,"71,447 miles","8.7 mi - Stroudsburg, PA","$21,990",High Price,"Black exterior, Brown interior","No accidents, 6 Owners, Personal use",yes,no,no,E92,Black,Brown
15,2010,335i Convertible,"74,799 miles","8.7 mi - Stroudsburg, PA","$18,990",High Price,"Black exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,no,E92,Black,Black
17,2011,335is Convertible,"40,262 miles","8.7 mi - Stroudsburg, PA","$28,990",High Price,"Blue exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,yes,E92,Blue,Black
18,2011,335i xDrive Coupe AWD,"86,645 miles","67 mi - Elizabeth, NJ","$19,895",Fair Price,"White exterior, Red interior","No accidents, 1 Owner, Personal use",yes,yes,no,E92,White,Red
20,2011,335i xDrive Coupe AWD,"34,908 miles","74 mi - Jersey City, NJ","$19,998",Excellent Price,"Blue exterior, Unknown interior","No accidents, 2 Owners, Personal use",yes,yes,no,E92,Blue,Unknown
21,2011,335i Convertible,"89,479 miles","67 mi - Elizabeth, NJ","$17,495",Fair Price,"Blue exterior, Beige interior","1 accident, 2 Owners, Personal use",yes,no,no,E92,Blue,Beige


### Accident Check

In [24]:
truecar_df['accident'] = truecar_df['History'].apply(lambda x: 'No' if 'no accidents' in x.lower() else 'Yes')

In [25]:
truecar_df.head(50)

,Year,Model,Mileage,Location,Price,Site Price Type,Color Scheme,History,335i_yn,xdrive_yn,is_model_yn,body_style,car_color,interior_color,accident
0,2011,335is Convertible,"62,515 miles","58 mi - Somerset, NJ","$25,999",High Price,"Silver exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,yes,E92,Silver,Black,No
1,2011,335i xDrive Coupe AWD,"37,739 miles","21 mi - Easton, PA","$25,998",Excellent Price,"White exterior, Brown interior","No accidents, 2 Owners, Personal use",yes,yes,no,E92,White,Brown,No
4,2011,335i Sedan,"67,840 miles","8.7 mi - Stroudsburg, PA","$18,990",Great Price,"Blue exterior, Brown interior","No accidents, 5 Owners, Fleet use",yes,no,no,E90,Blue,Brown,No
8,2011,335i Convertible,"55,578 miles","21 mi - Easton, PA","$23,998",High Price,"Black exterior, Unknown interior","No accidents, 2 Owners, Personal use",yes,no,no,E92,Black,Unknown,No
11,2011,335i Convertible,"71,447 miles","8.7 mi - Stroudsburg, PA","$21,990",High Price,"Black exterior, Brown interior","No accidents, 6 Owners, Personal use",yes,no,no,E92,Black,Brown,No
15,2010,335i Convertible,"74,799 miles","8.7 mi - Stroudsburg, PA","$18,990",High Price,"Black exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,no,E92,Black,Black,No
17,2011,335is Convertible,"40,262 miles","8.7 mi - Stroudsburg, PA","$28,990",High Price,"Blue exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,yes,E92,Blue,Black,No
18,2011,335i xDrive Coupe AWD,"86,645 miles","67 mi - Elizabeth, NJ","$19,895",Fair Price,"White exterior, Red interior","No accidents, 1 Owner, Personal use",yes,yes,no,E92,White,Red,No
20,2011,335i xDrive Coupe AWD,"34,908 miles","74 mi - Jersey City, NJ","$19,998",Excellent Price,"Blue exterior, Unknown interior","No accidents, 2 Owners, Personal use",yes,yes,no,E92,Blue,Unknown,No
21,2011,335i Convertible,"89,479 miles","67 mi - Elizabeth, NJ","$17,495",Fair Price,"Blue exterior, Beige interior","1 accident, 2 Owners, Personal use",yes,no,no,E92,Blue,Beige,Yes


### Remove "miles" in Mileage


In [26]:
truecar_df['Mileage'] = truecar_df['Mileage'].apply(lambda x: x.split('m')[0])

In [27]:
truecar_df.head(5)

,Year,Model,Mileage,Location,Price,Site Price Type,Color Scheme,History,335i_yn,xdrive_yn,is_model_yn,body_style,car_color,interior_color,accident
0,2011,335is Convertible,"62,515","58 mi - Somerset, NJ","$25,999",High Price,"Silver exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,yes,E92,Silver,Black,No
1,2011,335i xDrive Coupe AWD,"37,739","21 mi - Easton, PA","$25,998",Excellent Price,"White exterior, Brown interior","No accidents, 2 Owners, Personal use",yes,yes,no,E92,White,Brown,No
4,2011,335i Sedan,"67,840","8.7 mi - Stroudsburg, PA","$18,990",Great Price,"Blue exterior, Brown interior","No accidents, 5 Owners, Fleet use",yes,no,no,E90,Blue,Brown,No
8,2011,335i Convertible,"55,578","21 mi - Easton, PA","$23,998",High Price,"Black exterior, Unknown interior","No accidents, 2 Owners, Personal use",yes,no,no,E92,Black,Unknown,No
11,2011,335i Convertible,"71,447","8.7 mi - Stroudsburg, PA","$21,990",High Price,"Black exterior, Brown interior","No accidents, 6 Owners, Personal use",yes,no,no,E92,Black,Brown,No


###  Removing miles in Location

In [28]:
truecar_df['Location'] = truecar_df['Location'].apply(lambda x: x.split('-')[1])

In [29]:
truecar_df.head(40)

,Year,Model,Mileage,Location,Price,Site Price Type,Color Scheme,History,335i_yn,xdrive_yn,is_model_yn,body_style,car_color,interior_color,accident
0,2011,335is Convertible,"62,515","Somerset, NJ","$25,999",High Price,"Silver exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,yes,E92,Silver,Black,No
1,2011,335i xDrive Coupe AWD,"37,739","Easton, PA","$25,998",Excellent Price,"White exterior, Brown interior","No accidents, 2 Owners, Personal use",yes,yes,no,E92,White,Brown,No
4,2011,335i Sedan,"67,840","Stroudsburg, PA","$18,990",Great Price,"Blue exterior, Brown interior","No accidents, 5 Owners, Fleet use",yes,no,no,E90,Blue,Brown,No
8,2011,335i Convertible,"55,578","Easton, PA","$23,998",High Price,"Black exterior, Unknown interior","No accidents, 2 Owners, Personal use",yes,no,no,E92,Black,Unknown,No
11,2011,335i Convertible,"71,447","Stroudsburg, PA","$21,990",High Price,"Black exterior, Brown interior","No accidents, 6 Owners, Personal use",yes,no,no,E92,Black,Brown,No
15,2010,335i Convertible,"74,799","Stroudsburg, PA","$18,990",High Price,"Black exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,no,E92,Black,Black,No
17,2011,335is Convertible,"40,262","Stroudsburg, PA","$28,990",High Price,"Blue exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,yes,E92,Blue,Black,No
18,2011,335i xDrive Coupe AWD,"86,645","Elizabeth, NJ","$19,895",Fair Price,"White exterior, Red interior","No accidents, 1 Owner, Personal use",yes,yes,no,E92,White,Red,No
20,2011,335i xDrive Coupe AWD,"34,908","Jersey City, NJ","$19,998",Excellent Price,"Blue exterior, Unknown interior","No accidents, 2 Owners, Personal use",yes,yes,no,E92,Blue,Unknown,No
21,2011,335i Convertible,"89,479","Elizabeth, NJ","$17,495",Fair Price,"Blue exterior, Beige interior","1 accident, 2 Owners, Personal use",yes,no,no,E92,Blue,Beige,Yes


### Number of accidents (Not working, will revisit)

In [41]:
# truecar_df['num_accidents'] = truecar_df['History'].apply(lambda x: x[0] if truecar_df['accident'] == 'Yes' else 'No')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

### Number Of Owners

In [31]:
truecar_df['num_owners'] = truecar_df['History'].apply(lambda x: x.split(',')[1])

In [32]:
truecar_df.head(55)

,Year,Model,Mileage,Location,Price,Site Price Type,Color Scheme,History,335i_yn,xdrive_yn,is_model_yn,body_style,car_color,interior_color,accident,num_owners
0,2011,335is Convertible,"62,515","Somerset, NJ","$25,999",High Price,"Silver exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,yes,E92,Silver,Black,No,3 Owners
1,2011,335i xDrive Coupe AWD,"37,739","Easton, PA","$25,998",Excellent Price,"White exterior, Brown interior","No accidents, 2 Owners, Personal use",yes,yes,no,E92,White,Brown,No,2 Owners
4,2011,335i Sedan,"67,840","Stroudsburg, PA","$18,990",Great Price,"Blue exterior, Brown interior","No accidents, 5 Owners, Fleet use",yes,no,no,E90,Blue,Brown,No,5 Owners
8,2011,335i Convertible,"55,578","Easton, PA","$23,998",High Price,"Black exterior, Unknown interior","No accidents, 2 Owners, Personal use",yes,no,no,E92,Black,Unknown,No,2 Owners
11,2011,335i Convertible,"71,447","Stroudsburg, PA","$21,990",High Price,"Black exterior, Brown interior","No accidents, 6 Owners, Personal use",yes,no,no,E92,Black,Brown,No,6 Owners
15,2010,335i Convertible,"74,799","Stroudsburg, PA","$18,990",High Price,"Black exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,no,E92,Black,Black,No,3 Owners
17,2011,335is Convertible,"40,262","Stroudsburg, PA","$28,990",High Price,"Blue exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,yes,E92,Blue,Black,No,3 Owners
18,2011,335i xDrive Coupe AWD,"86,645","Elizabeth, NJ","$19,895",Fair Price,"White exterior, Red interior","No accidents, 1 Owner, Personal use",yes,yes,no,E92,White,Red,No,1 Owner
20,2011,335i xDrive Coupe AWD,"34,908","Jersey City, NJ","$19,998",Excellent Price,"Blue exterior, Unknown interior","No accidents, 2 Owners, Personal use",yes,yes,no,E92,Blue,Unknown,No,2 Owners
21,2011,335i Convertible,"89,479","Elizabeth, NJ","$17,495",Fair Price,"Blue exterior, Beige interior","1 accident, 2 Owners, Personal use",yes,no,no,E92,Blue,Beige,Yes,2 Owners


#### Further cleaning

In [33]:
truecar_df['num_owners'] = truecar_df['num_owners'].apply(lambda x: x.replace('Owners', ''))

In [34]:
truecar_df.head(25)

,Year,Model,Mileage,Location,Price,Site Price Type,Color Scheme,History,335i_yn,xdrive_yn,is_model_yn,body_style,car_color,interior_color,accident,num_owners
0,2011,335is Convertible,"62,515","Somerset, NJ","$25,999",High Price,"Silver exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,yes,E92,Silver,Black,No,3
1,2011,335i xDrive Coupe AWD,"37,739","Easton, PA","$25,998",Excellent Price,"White exterior, Brown interior","No accidents, 2 Owners, Personal use",yes,yes,no,E92,White,Brown,No,2
4,2011,335i Sedan,"67,840","Stroudsburg, PA","$18,990",Great Price,"Blue exterior, Brown interior","No accidents, 5 Owners, Fleet use",yes,no,no,E90,Blue,Brown,No,5
8,2011,335i Convertible,"55,578","Easton, PA","$23,998",High Price,"Black exterior, Unknown interior","No accidents, 2 Owners, Personal use",yes,no,no,E92,Black,Unknown,No,2
11,2011,335i Convertible,"71,447","Stroudsburg, PA","$21,990",High Price,"Black exterior, Brown interior","No accidents, 6 Owners, Personal use",yes,no,no,E92,Black,Brown,No,6
15,2010,335i Convertible,"74,799","Stroudsburg, PA","$18,990",High Price,"Black exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,no,E92,Black,Black,No,3
17,2011,335is Convertible,"40,262","Stroudsburg, PA","$28,990",High Price,"Blue exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,yes,E92,Blue,Black,No,3
18,2011,335i xDrive Coupe AWD,"86,645","Elizabeth, NJ","$19,895",Fair Price,"White exterior, Red interior","No accidents, 1 Owner, Personal use",yes,yes,no,E92,White,Red,No,1 Owner
20,2011,335i xDrive Coupe AWD,"34,908","Jersey City, NJ","$19,998",Excellent Price,"Blue exterior, Unknown interior","No accidents, 2 Owners, Personal use",yes,yes,no,E92,Blue,Unknown,No,2
21,2011,335i Convertible,"89,479","Elizabeth, NJ","$17,495",Fair Price,"Blue exterior, Beige interior","1 accident, 2 Owners, Personal use",yes,no,no,E92,Blue,Beige,Yes,2


### State

In [35]:
truecar_df['State'] = truecar_df['Location'].apply(lambda x: x.split(',')[1])

In [36]:
truecar_df.head(50)

,Year,Model,Mileage,Location,Price,Site Price Type,Color Scheme,History,335i_yn,xdrive_yn,is_model_yn,body_style,car_color,interior_color,accident,num_owners,State
0,2011,335is Convertible,"62,515","Somerset, NJ","$25,999",High Price,"Silver exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,yes,E92,Silver,Black,No,3,NJ
1,2011,335i xDrive Coupe AWD,"37,739","Easton, PA","$25,998",Excellent Price,"White exterior, Brown interior","No accidents, 2 Owners, Personal use",yes,yes,no,E92,White,Brown,No,2,PA
4,2011,335i Sedan,"67,840","Stroudsburg, PA","$18,990",Great Price,"Blue exterior, Brown interior","No accidents, 5 Owners, Fleet use",yes,no,no,E90,Blue,Brown,No,5,PA
8,2011,335i Convertible,"55,578","Easton, PA","$23,998",High Price,"Black exterior, Unknown interior","No accidents, 2 Owners, Personal use",yes,no,no,E92,Black,Unknown,No,2,PA
11,2011,335i Convertible,"71,447","Stroudsburg, PA","$21,990",High Price,"Black exterior, Brown interior","No accidents, 6 Owners, Personal use",yes,no,no,E92,Black,Brown,No,6,PA
15,2010,335i Convertible,"74,799","Stroudsburg, PA","$18,990",High Price,"Black exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,no,E92,Black,Black,No,3,PA
17,2011,335is Convertible,"40,262","Stroudsburg, PA","$28,990",High Price,"Blue exterior, Black interior","No accidents, 3 Owners, Personal use",yes,no,yes,E92,Blue,Black,No,3,PA
18,2011,335i xDrive Coupe AWD,"86,645","Elizabeth, NJ","$19,895",Fair Price,"White exterior, Red interior","No accidents, 1 Owner, Personal use",yes,yes,no,E92,White,Red,No,1 Owner,NJ
20,2011,335i xDrive Coupe AWD,"34,908","Jersey City, NJ","$19,998",Excellent Price,"Blue exterior, Unknown interior","No accidents, 2 Owners, Personal use",yes,yes,no,E92,Blue,Unknown,No,2,NJ
21,2011,335i Convertible,"89,479","Elizabeth, NJ","$17,495",Fair Price,"Blue exterior, Beige interior","1 accident, 2 Owners, Personal use",yes,no,no,E92,Blue,Beige,Yes,2,NJ


### Droping uncessary columns

In [37]:
truecar_df.drop(['Color Scheme', 'History','335i_yn'], axis = 1)

,Year,Model,Mileage,Location,Price,Site Price Type,xdrive_yn,is_model_yn,body_style,car_color,interior_color,accident,num_owners,State
0,2011,335is Convertible,"62,515","Somerset, NJ","$25,999",High Price,no,yes,E92,Silver,Black,No,3,NJ
1,2011,335i xDrive Coupe AWD,"37,739","Easton, PA","$25,998",Excellent Price,yes,no,E92,White,Brown,No,2,PA
4,2011,335i Sedan,"67,840","Stroudsburg, PA","$18,990",Great Price,no,no,E90,Blue,Brown,No,5,PA
8,2011,335i Convertible,"55,578","Easton, PA","$23,998",High Price,no,no,E92,Black,Unknown,No,2,PA
11,2011,335i Convertible,"71,447","Stroudsburg, PA","$21,990",High Price,no,no,E92,Black,Brown,No,6,PA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919,2011,335i xDrive Coupe AWD,"34,908","Jersey City, NJ","$19,998",Excellent Price,yes,no,E92,Blue,Unknown,No,2,NJ
920,2011,335i Convertible,"89,479","Elizabeth, NJ","$17,495",Fair Price,no,no,E92,Blue,Beige,Yes,2,NJ
922,2008,335i Convertible,"137,855","Huntingdon Valley, PA","$11,295",High Price,no,no,E92,Black,Beige,No,5,PA
923,2009,335i Convertible,"60,798","Stroudsburg, PA","$20,990",High Price,no,no,E92,Black,Brown,No,3,PA


### Output to CSV (Commented out so file isn't generated everytime script runs)

In [38]:
# truecar_df.to_csv('car_data_cleaned.csv', index = False)

## Exploratory Data Analysis